In [1]:
import xarray 
import os
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import imdlib as imd
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
import matplotlib.image as mpimg
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [2]:
X_train = pd.read_csv(r"C:\Users\HP\Desktop\COLLEGE WORK\SEM 5\Machine Learning\Climate data modelling\Clustering algorithms\Rain Category 1901-2011.csv",index_col=0)
y_train = pd.read_csv(r"C:\Users\HP\Desktop\COLLEGE WORK\SEM 5\Machine Learning\Climate data modelling\Clustering algorithms\SSTA Cluster Number 1901-2010 with 8 clusters.csv",index_col=0)
X_test = pd.read_csv(r"C:\Users\HP\Desktop\COLLEGE WORK\SEM 5\Machine Learning\Climate data modelling\Clustering algorithms\Rain Category 2011-2023.csv",index_col=0)
y_test = pd.read_csv(r"C:\Users\HP\Desktop\COLLEGE WORK\SEM 5\Machine Learning\Climate data modelling\Clustering algorithms\SSTA Cluster Number 2011-2022 with 8 clusters.csv",index_col=0)

In [3]:
X_train

,zone1,zone2,zone3,zone4,zone5,zone6
0,3,3,4,4,4,4
1,4,4,4,4,3,4
2,4,4,5,4,4,4
3,3,4,3,4,3,3
4,3,4,4,4,3,2
...,...,...,...,...,...,...
105,5,4,4,3,4,5
106,4,4,6,5,4,5
107,4,5,4,4,4,3
108,4,3,4,4,3,4


In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((110, 6), (110, 1), (12, 6), (12, 1))

In [5]:
y_train_array=y_train.values.ravel()
y_train_array

array([4, 7, 7, 0, 7, 4, 3, 0, 0, 0, 0, 7, 4, 7, 5, 3, 0, 0, 5, 6, 3, 4,
       4, 4, 3, 6, 3, 4, 4, 4, 5, 4, 3, 3, 3, 4, 2, 3, 4, 5, 1, 5, 3, 6,
       2, 5, 6, 4, 3, 3, 4, 5, 5, 4, 3, 3, 5, 5, 6, 5, 6, 6, 6, 6, 4, 5,
       6, 3, 1, 1, 6, 6, 1, 6, 3, 4, 6, 5, 5, 1, 5, 1, 1, 2, 2, 6, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 6, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
      dtype=int64)

# Using hyper perameter for classification

In [6]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [7]:
model_params = {
    'svm': {
        'model': svm.SVC(decision_function_shape='ovo',gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'criterion': ['gini','entropy'],
            
        }
    },'linear_regression': {
        'model': LinearRegression(),
        'params': {}
    
    },
    'linear_discriminant_analysis': {
        'model': LinearDiscriminantAnalysis(),
        'params': {}
    
    },
}

In [8]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train_array)
    y_pred = clf.predict(X_test)
    y_pred = np.ceil(y_pred)
    y_test_array= y_test.values.ravel()
    y_test_array
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'accuracy_score': accuracy_score(y_pred,y_test_array),
        'best_params': clf.best_params_,
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','accuracy_score','best_params'])
df

,model,best_score,accuracy_score,best_params
0,svm,0.218182,0.750000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.290909,0.416667,{'n_estimators': 5}
2,logistic_regression,0.154545,0.583333,{'C': 1}
3,naive_bayes_gaussian,0.118182,0.083333,{}
4,naive_bayes_multinomial,0.236364,0.750000,{}
5,decision_tree,0.227273,0.250000,{'criterion': 'gini'}
6,linear_regression,-1.693394,0.000000,{}
7,linear_discriminant_analysis,0.163636,0.583333,{}
